Huggingface의 NLP tutorial 공부  
- 출처: https://huggingface.co/learn/nlp-course/chapter2/3?fw=pt
![huggingface](https://huggingface.co/front/assets/huggingface_logo-noborder.svg)

# Models

여기서는 hugging face의 model을 생성하고 사용하는 것을 알아볼 것이다.  
여기서 사용하는 클래스는 `AutoModel`로, 어떤 모델이던 checkpoint로부터 초기화해서 사용하기 편한 클래스이다.

AutoModel 클래스와 관련된 것들은 이 라이브러리의 사용가능한 다양한 모델들에 덮어씌우기 간단하다.  
하지만, 우리가 사용하고 싶은 모델의 타입을 안다면, 우리는 그 모델의 architecture를 정의한 클래스를 사용할 수 있다.  
여기서는 AutoModel의 뒤에서 어떠한 일이 일어나는지를 볼 것이다.

AutoModel의 과정은
1. 모델 불러오기
2. 모델에 pre trained weight 덮어씌우기

이번 tutorial에서는 BERT모델을 사용할 것이다.

## 1. Creating a Transformer

In [4]:
from transformers import BertConfig, BertModel

# Bert 설정 구성요소
config = BertConfig()
print(config)

model = BertModel(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



## 2. Different loading methods

In [5]:
from transformers import BertConfig, BertModel

config = BertConfig()
model = BertModel(config)

위 model 상태는 randomly initiated weight를 가진 BERT이다. 이 상태로도 사용할 수는 있겠지만 output이 말도 안되는, nonsence한 값일 것이다.

사용을 위해선 반드시 training을 해야한다. scratch로 우리가 직접 학습을 할 수 있지만, Chapter 1에서 본 것처럼 매우 많은 양의 데이터와 시간을 요구한다.  
그리고 무시할 수 없는 환경의 영향도 있다. 이러한 불필요하면서도 반복되는 작업을 피하기 위해, pre-trained model을 공유하고 사용하는 것은 불가피하다.

pre-trained model을 불러오자. 이를 위해서는 `from_pretrained` method를 사용한다.

In [6]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

이 작업은 우리가 이전에 보았던 `AutoModel` class와 동일한 작업이다. 즉, AutoClass로 위 작업을 대체할 수 있다.  
이제부터는 체크포인트에 상관없는 코드가 생성되기에, 한 체크포인트에서 작동하는 코드라면 다른 체크포인트에서도 작동한다.  
이는 architecture가 다르더라도 similar한 task에 대해서 학습된 것이면 적용될 수 있다고 한다.

위 코드에서 우리는 BertConf를 사용하지 않고 `bert-base-cased`를 통해서 pre-trained model을 불러왔다. 이는 BERT 논문 저자에 의해 학습된 모델 체크포인트이다. 더 자세한 것은 [model card](https://huggingface.co/bert-base-cased) 에서 확인할 수 있다.

이 모델은 checkpoint의 weight로 initialized 되었다.  
이제 이 모델은 학습되었던 task의 추론을 위해 사용이 가능하고, 새로운 task에 대해서 fine tuning역시 할 수 있다.  
scratch가 아닌 pre trained weight를 통해 학습함으로써 빠르게 좋은 성능을 달성할 수 있다.

weight는 다운받아지고 cash화 되어서 cash folder에 담겨진다.
- default는 ~/.cache/huggingface/transformers 이다. cash folder는 우리가 HF_HOME environment 변수를 setting해서 customize할 수 있다.

</br>

pre-trained model을 불러올 때 사용한 identifier(예: `bert-base-cased`)는 Model Hub의 어떠한 모델이라도 BERT architecture에 호환만 된다면 모델의 identifier가 될 수 있다. 
- 가능한 BERT checkpoint의 전체 list는 [여기서](https://huggingface.co/models?other=bert) 확인 가능

## 3. Saving methods

모델 저장은 모델 load만큼이나 쉽다. `save_pretrained()`를 사용하면 된다.

In [7]:
model.save_pretrained("directory_on_my_computer")

In [8]:
!ls directory_on_my_computer

config.json  model.safetensors


`config.json`은 model arcutecture build에 필요한 attributes를 담고있다.   
이는 checkpoint가 어디서 왔는지, 마지막 checkpoint에서 사용한 Trasnformers의 버전 무엇인지와 같은 몇몇 metadata도 담고있다.

`pytorch_model.bin`은 state dictionary로 알려져있다. 이는 모델의 모든 weights를 담고있다.

## 4. Inference를 위해 Trasnformer 모델 사용하기

이제 어떻게 모델을 load하고 save하는지 배웠다. 이제 실제로 모델을 사용해서 prediction하는 것을 배워보자.  
Transformer 모델은 숫자만을 다룰 수 있다. 여기서 숫자는 tokenizer가 만드는 것이다. 하지만 tokenizer를 얘기하지 전에, model의 input부터 보자.

Tokenizer는 input을 tensor에 캐스팅(?)할 수 있지만, 어떻게 작동하는지 이해를 하고자 input을 모델로 보내기 전에 어떤 작업이 완료되어야 하는지 볼 것이다.

In [9]:
sequences = ["Hello!", "Cool", "Nice!"]

Tokenizer는 이들을 vocabulary indices로 변환한다.  
각 sequence는 이제 숫자의 list이다. output은 다음과 같다.

In [10]:
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

In [11]:
import torch

model_inputs = torch.tensor(encoded_sequences)
model_inputs

tensor([[ 101, 7592,  999,  102],
        [ 101, 4658, 1012,  102],
        [ 101, 3835,  999,  102]])

## 5. Tensor를 model input으로 사용하기

In [12]:
output = model(model_inputs)

In [17]:
output[0].shape

torch.Size([3, 4, 768])

In [18]:
model_inputs.shape

torch.Size([3, 4])